In [11]:
import requests
from bs4 import BeautifulSoup
import re

In [30]:
regions = {"IDF": "12", "PACA": "21", "Aquitaine": "2"}

In [31]:
 def get_regions_cars(region):
    url="https://www.leboncoin.fr/recherche/?category=2&text=renault%20zoe&regions="+regions[region]
    req_headers = {
           "Accept-Language": "en-US,en;q=0.5",
           "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:40.0) Gecko/20100101 Firefox/40.0",
           "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
           "Referer": "http://thewebsite.com",
           "Connection": "keep-alive"
       }

    result = requests.get(url, headers=req_headers)
    soup = BeautifulSoup(result.text, 'html.parser')

    suffixes = [ad.attrs['href'] for ad in soup.findAll("a", class_ = "clearfix trackable")]
    return suffixes

In [32]:
import pandas as pd

In [65]:
df = pd.DataFrame(columns=["version", "année", "kilométrage", "prix", "téléphone", "vendeur"])

In [34]:
argus = {"intens":8277, "life": 7855, "zen":8131}

In [35]:
def add_ads(suff):
    res = requests.get("https://www.leboncoin.fr"+ suff, headers = req_headers)
    soup = BeautifulSoup(res.text, 'html.parser')
    #     info = soup.findAll("div", class_ = "mainInfos hiddenOverflow")
    #     km = soup.find("span", class_ = "fieldPrice sizeC")
    #     print(info)
    prix = soup.find("div", attrs = {"data-qa-id":"adview_price"})
    km = soup.find("div", attrs = {"data-qa-id":"criteria_item_mileage"})
#     modele = soup.find("div", attrs = {"data-qa-id":"criteria_item_model"})
    annee = soup.find("div", attrs = {"data-qa-id":"criteria_item_regdate"})
    vendeur = soup.find("div", attrs = {"data-qa-id":"adview_contact_container"})
    titre = soup.find("div", attrs = {"data-qa-id":"adview_title"})
    types = ["zen", "life", "intens"]
#     print(titre.text.lower())
    if "siret" in vendeur.text.lower():
        vendeur = "pro"
    else:
        vendeur = "particulier"
    try:
        modele = types[[modele in titre.text.lower() for modele in types].index(True)]
    except:
        modele = None
    try:
        arg = argus[modele]
    except:
        arg = None
    return pd.DataFrame({"version": modele, "année": annee.text,"kilométrage": km.text, "prix": prix.text, "vendeur":vendeur, "argus":arg}, index=[int(re.findall(r'\d+', suff)[0])])

In [56]:
def get_region_df(region):
    suffixes = get_regions_cars(region)
    global df
    df = {}
    df[region] = pd.concat([add_ads(suff) for suff in suffixes])

    df[region]["annee"] = df[region].année.str.extract('(\d+)')
    df[region]["km"] = df[region].kilométrage.str.extract('(\d+)')

    df[region]["prix_Euro"] = df[region].prix.str.extract("(\d+ \d{3})")

    df[region]["prix_Euro"] = df[region].prix_Euro.str.replace(" ","").astype(int)

    df[region] = df[region].dropna()

    df[region]= df[region].drop(columns=["année", "kilométrage","prix"])

    df[region]["compare"] = "="
    df[region]["compare"] = df[region]["compare"].where(df[region].prix_Euro>df[region].argus,"-")
    df[region]["compare"] = df[region]["compare"].where(df[region].prix_Euro<df[region].argus,"+")

In [57]:
%%time
for region in regions.keys():
    get_region_df(region)

CPU times: user 4.96 s, sys: 168 ms, total: 5.13 s
Wall time: 33.3 s


In [62]:
import multiprocessing.dummy as mp 

In [ ]:
%%time
p=mp.Pool(6)
p.map(get_region_df,regions.keys())
p.close()
p.join()

In [67]:
df['Aquitaine']

,version,vendeur,argus,annee,km,prix_Euro,compare
1513099194,life,particulier,7855,2014,58200,7500,-
1513182441,intens,pro,8277,2015,3120,10990,+
1512190101,life,pro,7855,2015,14908,9350,+
1512192311,life,particulier,7855,2016,3690,9990,+
1511162920,life,pro,7855,2014,48008,8200,+
1510599803,life,particulier,7855,2013,43000,7990,+
1510621539,zen,pro,8131,2018,100,21900,+
1509527811,life,pro,7855,2015,38501,8950,+
1509527909,life,pro,7855,2015,13798,9750,+
1509083121,zen,pro,8131,2015,27799,9400,+
